# Data cleaning and reformating

In [91]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import numpy as np
import os

from random import shuffle, seed

In [92]:
header = ['TimeSec', 'Sensor', 'Quatx', 'Quaty', 'Quatz', 'Quat0', 'Heading',
       'Pitch', 'Roll', 'LinAccx', 'LinAccy', 'LinAccz', 'Vbat', 'Accx',
       'Accy', 'Accz', 'Gyrox', 'Gyroy', 'Gyroz']

In [93]:
### Take a look at one dataset
file_path = "./spinetrack data/Task_Drilling_Stoop.csv"
df = pd.read_csv(file_path, error_bad_lines=False)
df.columns = header

df["Activity"] = [1 for i in range(len(df))]
df

b'Skipping line 1584: expected 19 fields, saw 24\nSkipping line 1898: expected 19 fields, saw 22\nSkipping line 1936: expected 19 fields, saw 27\n'


,TimeSec,Sensor,Quatx,Quaty,Quatz,Quat0,Heading,Pitch,Roll,LinAccx,LinAccy,LinAccz,Vbat,Accx,Accy,Accz,Gyrox,Gyroy,Gyroz,Activity
0,0.052342,7,-0.994,-0.01,-0.02,0.11,1.65,2.17,0.54,-0.01,-0.01,0.00,0.00,0.31,-0.15,1.53,-0.05,0.02,1.00,1
1,0.104450,2,0.801,0.05,0.02,-0.60,300.85,5.35,3.32,0.02,0.00,-0.01,3.89,3.51,-2.90,6.41,-0.08,0.08,0.98,1
2,0.106090,3,-0.781,0.05,0.03,-0.62,91.09,0.63,-7.05,0.07,0.00,0.00,3.75,-3.20,-1.22,0.00,0.00,-0.19,0.98,1
3,0.112490,5,-0.889,-0.07,-0.20,-0.41,66.01,17.11,17.30,0.03,0.02,0.01,3.80,-5.80,2.14,5.80,-0.29,0.26,0.93,1
4,0.114330,6,0.986,0.08,-0.02,-0.14,357.40,-1.53,9.05,-0.03,-0.01,0.01,3.75,0.46,0.31,0.15,-0.01,0.16,1.00,1
5,0.118730,4,-0.924,0.13,-0.13,0.34,332.62,8.14,-19.47,-0.06,0.02,0.00,3.73,6.56,-3.20,-2.90,-0.17,-0.38,0.92,1
6,0.120400,0,0.232,0.05,-0.05,-0.97,221.20,3.86,7.19,0.03,0.00,0.00,3.87,-0.31,3.05,-1.68,-0.09,0.14,0.99,1
7,0.146600,1,0.202,0.23,0.02,-0.95,218.87,27.04,3.36,-0.01,-0.02,0.01,3.89,0.61,1.98,-0.15,-0.45,0.03,0.90,1
8,0.148450,7,-0.994,-0.01,-0.02,0.11,1.75,2.17,0.57,-0.00,-0.01,-0.00,0.00,0.31,-0.61,-1.98,-0.05,0.02,0.99,1
9,0.158300,2,0.804,0.05,0.02,-0.59,301.38,4.98,3.70,0.05,0.01,0.01,3.89,1.53,-2.90,-0.31,-0.07,0.11,1.00,1


In [321]:
"%s number of activity: %s" %("4", "5")

'4 number of activity: 5'

In [338]:
index_dict = {}
activity = 0
for i in sorted(os.listdir('./Spinetrack Data/data/Alex_data/Processed_redone')):
    if i.endswith(".csv") and i != 'super_features.csv': 
        act = i.split('.')[0]
        index_dict[act] = activity
        activity += 1

In [339]:
print(index_dict)

{'Carrying_10': 0, 'Carrying_20': 1, 'Carrying_5': 2, 'Crawling': 3, 'Crouching': 4, 'Kneeling': 5, 'L_Floor_60in_10': 6, 'L_Floor_60in_20': 7, 'L_Floor_60in_5': 8, 'L_Floor_Shoulder_10': 9, 'L_Floor_Shoulder_20': 10, 'L_Floor_Shoulder_5': 11, 'L_Floor_Waist_Squat_10': 12, 'L_Floor_Waist_Squat_20': 13, 'L_Floor_Waist_Squat_5': 14, 'L_Floor_Waist_Stoop_10': 15, 'L_Floor_Waist_Stoop_20': 16, 'L_Floor_Waist_Stoop_5': 17, 'L_OneH_L_10': 18, 'L_OneH_L_15': 19, 'L_OneH_L_5': 20, 'L_OneH_R_10': 21, 'L_OneH_R_15': 22, 'L_OneH_R_5': 23, 'L_Shoulder_Waist_Twist_10': 24, 'L_Shoulder_Waist_Twist_20': 25, 'L_Shoulder_Waist_Twist_5': 26, 'L_Shoulder_Waist_wo_10': 27, 'L_Shoulder_Waist_wo_20': 28, 'L_Shoulder_Waist_wo_5': 29, 'Overhead_Dynamic': 30, 'Overhead_Static': 31, 'Pulling_Clockwise': 32, 'Pulling_OneH_L': 33, 'Pulling_OneH_R': 34, 'Pushing_Clockwise': 35, 'Pushing_Counterclockwise': 36, 'Reaching_Close': 37, 'Reaching_Far': 38, 'Reaching_High': 39, 'Sitting': 40, 'Standing': 41, 'Static_Stoo

In [336]:
directory = "./Spinetrack Data/data/"
file = []

for d in sorted(os.listdir(directory)):
    if d != '.DS_Store':
        files = directory + d + "/Processed_redone"
        name = []
        try:
            for f in sorted(os.listdir(files)):
                if f.endswith(".csv") and f != 'super_features.csv': 
                    name.append(os.path.join(files, f))
            file.append(name)
            print("%s number of activity: %s" %(d, len(name)))
        except:
            pass
file

Alex_data number of activity: 44
Alexander_data number of activity: 43
Charlotte_data number of activity: 43
Christian_data number of activity: 34
Elias_data number of activity: 44
Jesse_data number of activity: 44
Jiyoo_data number of activity: 43


[['./Spinetrack Data/data/Alex_data/Processed_redone/Carrying_10.csv',
  './Spinetrack Data/data/Alex_data/Processed_redone/Carrying_20.csv',
  './Spinetrack Data/data/Alex_data/Processed_redone/Carrying_5.csv',
  './Spinetrack Data/data/Alex_data/Processed_redone/Crawling.csv',
  './Spinetrack Data/data/Alex_data/Processed_redone/Crouching.csv',
  './Spinetrack Data/data/Alex_data/Processed_redone/Kneeling.csv',
  './Spinetrack Data/data/Alex_data/Processed_redone/L_Floor_60in_10.csv',
  './Spinetrack Data/data/Alex_data/Processed_redone/L_Floor_60in_20.csv',
  './Spinetrack Data/data/Alex_data/Processed_redone/L_Floor_60in_5.csv',
  './Spinetrack Data/data/Alex_data/Processed_redone/L_Floor_Shoulder_10.csv',
  './Spinetrack Data/data/Alex_data/Processed_redone/L_Floor_Shoulder_20.csv',
  './Spinetrack Data/data/Alex_data/Processed_redone/L_Floor_Shoulder_5.csv',
  './Spinetrack Data/data/Alex_data/Processed_redone/L_Floor_Waist_Squat_10.csv',
  './Spinetrack Data/data/Alex_data/Proce

In [349]:
frame = []
for person in file:
    for file_path in person:
        activity_name = file_path.split("/")[-1].split('.')[0]
        if activity_name in index_dict.keys():
            csv_name = '/' + activity_name + '.csv'
            folder_name = file_path.split("/")[-3] # person's name 
            category = 'data/' # data or task
            # mkdir if not exist
            if not (os.path.exists('./Spinetrack Data/Yibin_Processed/' + category + folder_name)):
                os.makedirs('./Spinetrack Data/Yibin_Processed/' + category + folder_name)
            file_name = './Spinetrack Data/Yibin_Processed/' + category + folder_name + csv_name
            df = pd.read_csv(file_path, error_bad_lines=False)
            df.columns = header
            df["activity"] = [index_dict[activity_name] for i in range(len(df))]
            frame.append(df)
            #df.to_csv(file_name) # save csv processed file to local
            print(file_name)

./Spinetrack Data/Yibin_Processed/data/Alex_data/Carrying_10.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/Carrying_20.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/Carrying_5.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/Crawling.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/Crouching.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/Kneeling.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/L_Floor_60in_10.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/L_Floor_60in_20.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/L_Floor_60in_5.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/L_Floor_Shoulder_10.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/L_Floor_Shoulder_20.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/L_Floor_Shoulder_5.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/L_Floor_Waist_Squat_10.csv
./Spinetrack Data/Yibin_Processed/data/Alex_data/L_Floor_Waist_Squat_20.csv
./Spinetrack Data/Yibin_Processed/data/Alex

./Spinetrack Data/Yibin_Processed/data/Charlotte_data/Pushing_Counterclockwise.csv
./Spinetrack Data/Yibin_Processed/data/Charlotte_data/Reaching_Close.csv
./Spinetrack Data/Yibin_Processed/data/Charlotte_data/Reaching_Far.csv
./Spinetrack Data/Yibin_Processed/data/Charlotte_data/Reaching_High.csv
./Spinetrack Data/Yibin_Processed/data/Charlotte_data/Sitting.csv
./Spinetrack Data/Yibin_Processed/data/Charlotte_data/Standing.csv
./Spinetrack Data/Yibin_Processed/data/Charlotte_data/Static_Stoop.csv
./Spinetrack Data/Yibin_Processed/data/Charlotte_data/Walking.csv
./Spinetrack Data/Yibin_Processed/data/Christian_data/Crawling.csv
./Spinetrack Data/Yibin_Processed/data/Christian_data/Crouching.csv
./Spinetrack Data/Yibin_Processed/data/Christian_data/Kneeling.csv
./Spinetrack Data/Yibin_Processed/data/Christian_data/Overhead_Dynamic.csv
./Spinetrack Data/Yibin_Processed/data/Christian_data/Overhead_Static.csv
./Spinetrack Data/Yibin_Processed/data/Christian_data/Pulling_Clockwise.csv
./Sp

./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/L_Shoulder_Waist_Twist_5.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/L_Shoulder_Waist_wo_10.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/L_Shoulder_Waist_wo_20.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/L_Shoulder_Waist_wo_5.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/Overhead_Dynamic.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/Overhead_Static.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/Pulling_Clockwise.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/Pulling_OneH_L.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/Pulling_OneH_R.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/Pushing_Clockwise.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/Pushing_Counterclockwise.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/Reaching_Close.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/Reaching_Far.csv
./Spinetrack Data/Yibin_Processed/data/Jiyoo_data/Rea

In [350]:
data_df = pd.concat(frame)
data_df

,TimeSec,Sensor,Quatx,Quaty,Quatz,Quat0,Heading,Pitch,Roll,LinAccx,LinAccy,LinAccz,Vbat,Accx,Accy,Accz,Gyrox,Gyroy,Gyroz,activity
0,0.0362,2,0.822,0.01,0.26,-0.51,306.93,25.57,-16.37,-0.03,-0.03,-0.02,3.94,-19.84,-11.14,16.33,-0.47,-0.28,0.81,0
1,0.0377,3,0.625,-0.21,0.26,0.71,113.60,38.21,7.42,0.06,-0.08,-0.08,3.80,-37.84,-4.27,53.71,-0.51,0.03,0.77,0
2,0.0961,0,0.008,0.08,0.02,1.00,192.97,-9.34,2.16,0.11,-0.13,-0.02,3.92,2.75,-1.68,36.01,0.07,-0.10,0.98,0
3,0.0978,6,-0.978,0.03,-0.00,-0.21,38.13,1.13,-2.73,0.08,0.36,0.33,3.81,-18.92,-64.70,118.26,-0.13,0.07,1.20,0
4,0.1018,5,0.448,0.44,0.49,0.61,116.18,-5.60,82.09,0.26,-0.13,-0.21,3.87,5.19,45.62,-2.90,0.15,0.67,0.35,0
5,0.1032,1,0.105,-0.09,0.01,-0.99,206.37,-9.81,-2.63,-0.01,0.04,-0.05,3.91,-3.51,18.31,61.04,0.20,-0.02,0.84,0
6,0.1069,4,0.540,-0.59,0.39,-0.45,300.68,-6.56,-90.39,0.10,0.06,-0.26,3.79,8.85,-50.35,-32.35,0.15,-1.09,0.06,0
7,0.1086,7,0.955,0.11,0.16,0.23,42.78,15.05,16.65,0.11,0.29,-0.22,0.00,-14.34,-24.11,85.60,-0.28,-0.09,0.82,0
8,0.1283,2,0.838,-0.01,0.26,-0.48,310.55,24.85,-16.97,-0.12,0.06,-0.06,3.94,-31.13,-19.23,33.87,-0.37,-0.39,0.77,0
9,0.1301,3,0.586,-0.22,0.23,0.74,119.48,37.14,5.69,-0.01,0.01,-0.16,3.80,-56.00,1.37,29.91,-0.50,0.10,0.68,0


In [371]:
# result_df = alex_data_df
# result_df
result_df = data_df

In [271]:
# # the sensor label index range from 0 to 7 
# sensor_data = []
# for i in range(0, 8):
#     df = alex_data_df.where(alex_data_df['Sensor'] == i).dropna()
#     sensor_data.append(df)
# result_df = pd.concat(sensor_data).reset_index(drop=True)
# result_df

In [272]:
print(result_df[4350:4450])

     TimeSec  Sensor  Quatx  Quaty  Quatz  Quat0  Heading  Pitch    Roll  \
253    0.707       4 -0.335   0.55  -0.74   0.18   274.27  17.10 -137.86   
254    0.709       6  0.998  -0.05  -0.03   0.02    16.34  -3.59   -5.30   
255    0.715       5 -0.821   0.02  -0.19  -0.54    82.37  19.25    9.79   
256    0.716       7 -0.971  -0.13  -0.02  -0.20    37.25  -1.20   15.16   
257    0.740       0  0.045  -0.16  -0.06  -0.98   198.22 -18.42    6.54   
258    0.742       3 -0.781   0.11   0.00  -0.61    89.76   7.65   -9.95   
259    0.749       2 -0.632  -0.07  -0.16   0.75   272.46  18.66   -9.37   
260    0.751       1 -0.180   0.02   0.01   0.98   214.83  -2.07    0.94   
261    0.807       4 -0.340   0.54  -0.73   0.22   273.50  15.11 -134.11   
262    0.809       6  0.999  -0.05  -0.02   0.00    14.32  -2.64   -5.29   
263    0.814       5 -0.834   0.03  -0.17  -0.52    79.53  18.32    7.56   
264    0.816       7 -0.973  -0.13  -0.03  -0.19    36.08   0.12   15.34   
265    0.839

# Data processing and deep learning model

In [372]:
# Process original dataset, create windows (window_size samples(rows), about 1 second)
window_size = 60
data = []
window = 1
while window*window_size < len(result_df):
    data_window = result_df[(window - 1)*window_size:window*window_size]
    data.append(data_window.values)
    window += 1
data

[array([[ 0.0362,  2.    ,  0.822 , ..., -0.28  ,  0.81  ,  0.    ],
        [ 0.0377,  3.    ,  0.625 , ...,  0.03  ,  0.77  ,  0.    ],
        [ 0.0961,  0.    ,  0.008 , ..., -0.1   ,  0.98  ,  0.    ],
        ...,
        [ 0.7439,  3.    ,  0.208 , ...,  0.14  ,  0.89  ,  0.    ],
        [ 0.7621,  2.    ,  0.966 , ..., -0.36  ,  0.92  ,  0.    ],
        [ 0.7636,  6.    , -0.869 , ..., -0.04  ,  1.27  ,  0.    ]]),
 array([[ 0.8162,  5.    ,  0.103 , ...,  1.    ,  0.17  ,  0.    ],
        [ 0.8178,  1.    ,  0.576 , ..., -0.13  ,  1.04  ,  0.    ],
        [ 0.8232,  4.    ,  0.676 , ..., -0.87  ,  0.08  ,  0.    ],
        ...,
        [ 1.515 ,  1.    ,  0.817 , ..., -0.04  ,  1.2   ,  0.    ],
        [ 1.5204,  4.    ,  0.708 , ..., -1.08  ,  0.13  ,  0.    ],
        [ 1.5222,  7.    ,  0.45  , ...,  0.56  ,  1.29  ,  0.    ]]),
 array([[ 1.5455,  2.    ,  0.976 , ..., -0.38  ,  1.03  ,  0.    ],
        [ 1.5472,  3.    ,  0.01  , ..., -0.19  ,  0.71  ,  0.    ],
    

In [373]:
len(data)

14465

In [374]:
# delete window if multiple activities and sensors presents
cleaned_data = []
for i in data:
    previous_activity = -1
    previous_sensor = -1
    for j in i:
        current_activity = j[19]
        current_sensor = j[1]
        if (previous_activity != -1) and (current_activity != previous_activity):
            print("data contains different activities! Window droped")
            break
#         elif (previous_sensor != -1) and (current_sensor != previous_sensor):
#             print("data contains different sensors! Window droped")
#             break
        else:
            previous_activity = current_activity
            previous_sensor = current_sensor
    else:
        cleaned_data.append(i)

data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped


data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped


In [375]:
len(cleaned_data)

14193

In [376]:
cleaned_data[0].shape

(60, 20)

In [377]:
# shuffle the data
seed(101)
shuffle(cleaned_data)
cleaned_data

[array([[21.976,  2.   , -0.959, ...,  0.2  ,  0.81 ,  3.   ],
        [21.977,  4.   , -0.808, ...,  0.52 ,  1.61 ,  3.   ],
        [22.011,  3.   ,  0.375, ..., -0.67 ,  0.96 ,  3.   ],
        ...,
        [22.669,  4.   , -0.871, ...,  0.03 ,  1.31 ,  3.   ],
        [22.718,  1.   , -0.596, ...,  0.06 ,  0.74 ,  3.   ],
        [22.72 ,  5.   , -0.277, ..., -0.03 ,  0.98 ,  3.   ]]),
 array([[20.225,  7.   ,  0.889, ...,  0.04 ,  0.99 , 30.   ],
        [20.292,  4.   ,  0.443, ...,  0.59 , -0.36 , 30.   ],
        [20.293,  2.   ,  0.038, ..., -0.44 , -0.57 , 30.   ],
        ...,
        [21.   ,  1.   ,  0.591, ...,  0.   ,  0.   , 30.   ],
        [21.001,  2.   ,  0.204, ..., -0.13 , -0.52 , 30.   ],
        [21.005,  4.   ,  0.201, ...,  0.51 , -0.79 , 30.   ]]),
 array([[45.03 ,  2.   ,  0.9  , ..., -0.2  ,  0.94 , 37.   ],
        [45.055,  4.   ,  0.419, ..., -0.5  , -0.64 , 37.   ],
        [45.057,  3.   ,  0.619, ...,  0.31 ,  0.95 , 37.   ],
        ...,
        [45.

In [378]:
# extract label from each window
labels = []
for i in cleaned_data:
    label = i[0][19]
    labels.append(label)
labels = np.array(labels)
#labels

In [379]:
# extract features from each window
features = []
for i in cleaned_data:
    new = np.delete(i, 19, 1)
    features.append(new)
features = np.array(features)
#features

In [380]:
features[0].shape

(60, 19)

In [381]:
# combine the features and labels
k = list(zip(features, labels))
activity_data = pd.DataFrame(k)
activity_data.columns = ['features', 'labels']
activity_data

,features,labels
0,"[[21.976, 2.0, -0.9590000000000001, 0.11, 0.2,...",3.0
1,"[[20.225, 7.0, 0.889, 0.03, 0.01, -0.46, 319.7...",30.0
2,"[[45.03, 2.0, 0.9, -0.03, 0.18, -0.4, 324.62, ...",37.0
3,"[[21.326999999999998, 0.0, 0.971, -0.04, 0.01,...",34.0
4,"[[40.524, 6.0, -0.9740000000000001, -0.03, -0....",39.0
5,"[[2.5077, 2.0, -0.9229999999999999, -0.29, -0....",33.0
6,"[[0.9976, 1.0, 0.851, 0.16, 0.33, -0.38, 327.2...",4.0
7,"[[2.8160000000000003, 6.0, 0.40399999999999997...",10.0
8,"[[1.8359999999999999, 6.0, -0.9890000000000001...",13.0
9,"[[32.236999999999995, 7.0, -0.9790000000000001...",3.0


In [383]:
# check the size of activity. The final output of neural net 
# has to have max_index + 1 output

# max_index = activity_data['labels'].max()

In [384]:
# split the data to test and train
activity_data.columns = ["features", "labels"]
activity_data_train = activity_data[:int(len(activity_data)*0.7)]
activity_data_test = activity_data[int(len(activity_data)*0.7):]
# activity_data_train.to_csv("./activity_data_train.csv", encoding='utf-8-sig')
# activity_data_train.to_csv("./activity_data_test.csv", encoding='utf-8-sig')

In [385]:
# define our dataset in pytorch
class DatasetSpineTrack(Dataset):
    
    def __init__(self, file, transform=None):
        #self.data = pd.read_csv(file_path)
        self.data = file
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        
        features = torch.tensor(self.data["features"].iloc[index])
        features = features.view(window_size, 1, 19)
        labels = torch.tensor(self.data["labels"].iloc[index], dtype=torch.long)
        #print(labels.type())
        
#         if self.transform is not None:
#             image = self.transform(image)
            
        return features, labels

In [386]:
# construct training and testing dataset in csv
# train_dataset = DatasetSpineTrack("./activity_data_train.csv")
# test_dataset = DatasetSpineTrack("./activity_data_test.csv")
train_dataset = DatasetSpineTrack(activity_data_train)
test_dataset = DatasetSpineTrack(activity_data_test)
feature, label = train_dataset.__getitem__(0)
#feature
#label

In [387]:
# load data
trainloader = DataLoader(train_dataset, batch_size=64)
testloader = DataLoader(test_dataset, batch_size=64)

In [390]:
model = models.resnet18(pretrained=False)
model.conv1 = torch.nn.Conv2d(window_size, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(512, 44, bias=True)
model = model.double()
model

ResNet(
  (conv1): Conv2d(60, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): C

In [391]:
import time
start_time = time.time()

criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 30

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for feature, label in trainloader:
        
        optimizer.zero_grad()
        
        log_ps = model(feature)
        loss = criterion(log_ps, label)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        #print(running_loss)
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for feature, label in testloader:
                log_ps = model(feature)
                test_loss += criterion(log_ps, label)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == label.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

# output running time
running_time = time.time() - start_time
sec = running_time % 60
miniute = running_time / 60
print("Training time: {} min {} sec".format(int(miniute), sec))

Epoch: 1/30..  Training Loss: 2.667..  Test Loss: 2.198..  Test Accuracy: 0.313
Epoch: 2/30..  Training Loss: 1.880..  Test Loss: 1.847..  Test Accuracy: 0.389
Epoch: 3/30..  Training Loss: 1.584..  Test Loss: 1.725..  Test Accuracy: 0.429
Epoch: 4/30..  Training Loss: 1.391..  Test Loss: 1.748..  Test Accuracy: 0.440
Epoch: 5/30..  Training Loss: 1.287..  Test Loss: 1.693..  Test Accuracy: 0.472
Epoch: 6/30..  Training Loss: 1.184..  Test Loss: 1.628..  Test Accuracy: 0.496
Epoch: 7/30..  Training Loss: 1.113..  Test Loss: 1.631..  Test Accuracy: 0.504
Epoch: 8/30..  Training Loss: 1.014..  Test Loss: 1.648..  Test Accuracy: 0.521
Epoch: 9/30..  Training Loss: 0.925..  Test Loss: 1.699..  Test Accuracy: 0.515
Epoch: 10/30..  Training Loss: 0.850..  Test Loss: 1.777..  Test Accuracy: 0.522
Epoch: 11/30..  Training Loss: 0.802..  Test Loss: 1.807..  Test Accuracy: 0.529
Epoch: 12/30..  Training Loss: 0.769..  Test Loss: 1.860..  Test Accuracy: 0.522
Epoch: 13/30..  Training Loss: 0.717.

In [78]:
from torch import tensor 
o = tensor([[[ 1.0028, -0.9893,  0.5809],
        [-0.1669,  0.7299,  0.4942]], 
            [[2, 1, 3], 
             [4, 5, 6]]])
torch.transpose(o, 1, 2)

tensor([[[ 1.0028, -0.1669],
         [-0.9893,  0.7299],
         [ 0.5809,  0.4942]],

        [[ 2.0000,  4.0000],
         [ 1.0000,  5.0000],
         [ 3.0000,  6.0000]]])